# Solving the Phillips-Wunsch-Garrett IVP with non-constant coefficients using Dedalus

In [4]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import h5py
from dedalus import public as de
from dedalus.extras import flow_tools
from dedalus.extras import plot_tools
from dedalus.tools import post
import pathlib
import time
from IPython import display

import logging
root = logging.root
for h in root.handlers:
    h.setLevel("INFO")

logger = logging.getLogger(__name__)

# Problem formulation

### Physical parameters

In [5]:
N = 1.3e-3
f = 0.53e-4

# topographic parameters
slopeAngle = 2.e-3
tht = slopeAngle

# mixing parameters
d = 230
k0 = 5.2e-5
k1 = 1.8e-3
Pr = 1

In [6]:
def calc_delta():
    return ((4*Pr**2*(k0+k1)**2) / (f**2*np.cos(tht)**2 * (1 + (N**2*np.tan(tht)**2/(f**2))*Pr)))**0.25

In [9]:
#===== Set up domain =====
# Create basis and domain for all experiments
Lz = 4000
nz = 512
z_basis = de.Chebyshev('z', nz, interval=(0, Lz), dealias=3/2)
domain = de.Domain([z_basis],np.float64)
z = domain.grid(0)

#===== Set up Problem ======
problem = de.IVP(domain, variables=['u', 'v', 'b', 'uz', 'vz', 'bz']);

# Set parameters
problem.parameters['f'] = f
problem.parameters['tht'] = tht # set in loop
problem.parameters['N'] = N
k = domain.new_field(name='k')
k['g'] = k0+k1*np.exp(-z/d)
problem.parameters['k'] = k

nu = domain.new_field(name='nu')
nu['g'] = (k0+k1*np.exp(-z/d))*Pr
problem.parameters['nu'] = nu

# Main equations
problem.add_equation("-f*cos(tht)*v - sin(tht)*b - dz(nu*uz) = 0")
problem.add_equation("f*u*cos(tht) - dz(nu*vz) = 0")
problem.add_equation("dt(b) + N**2*sin(tht)*u - dz(k*bz) = N**2*cos(tht)*dz(k)")

# Auxiliary equations defining the first-order reduction
problem.add_equation("uz - dz(u) = 0")
problem.add_equation("vz - dz(v) = 0")
problem.add_equation("bz - dz(b) = 0")

# Boundary conditions
problem.add_bc('left(u) = 0')
problem.add_bc('left(v) = 0')
problem.add_bc('left(bz) = - N**2*cos(tht)')
problem.add_bc('right(uz) = 0')
problem.add_bc('right(vz) = 0')
problem.add_bc('right(bz) = 0')

# Set solver for IVP
solver = problem.build_solver(de.timesteppers.RK443);

#==== Set initial conditions ====
# Reference local grid and state fields
z = domain.grid(0)
u = solver.state['u']
v = solver.state['v']
b = solver.state['b']
uz = solver.state['uz']
vz = solver.state['vz']
bz = solver.state['bz']

# State from a state of rest
u['g'] = np.zeros_like(z)
v['g'] = np.zeros_like(z)
b['g'] = np.zeros_like(z)
u.differentiate('z', out=uz)
v.differentiate('z', out=vz)
b.differentiate('z', out=bz)


#==== Create analysis files ====
output_path = '../../data/dedalus/spinup'
analysis = solver.evaluator.add_file_handler(output_path, iter=1)
analysis.add_system(solver.state, layout='g')

# Stop stopping criteria
solver.stop_sim_time = (10.*24.*60.*60.)
solver.stop_wall_time = np.inf
solver.stop_iteration = np.inf

# Main loop
dt = 3.*60.*60.
start_time = time.time()
print('Years: ')
while solver.ok:
    solver.step(dt);
    if solver.sim_time % (3.*60.*60.) == 0:
        print(solver.sim_time/(3.*60.*60.), end=", ")
        
end_time = time.time()
print('Runtime:', end_time-start_time)
z_da = domain.grid(0, scales=domain.dealias)

2020-02-25 13:20:49,502 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 1.0e+01/s
Years: 
1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 2020-02-25 13:20:52,552 solvers 0/1 INFO :: Simulation stop time reached.
Runtime: 3.041898488998413


In [10]:
from dedalus.tools import post
import pathlib
import os
post.merge_process_files(output_path, cleanup=True)
set_paths = list(pathlib.Path(output_path).glob("spinup_s*.h5"))
import os
os.system(f"rm -f {output_path}/output.h5")
post.merge_sets(output_path+"/output.h5", set_paths, cleanup=True)

2020-02-25 13:21:04,589 post 0/1 INFO :: Merging files from ../../data/dedalus/spinup
2020-02-25 13:21:04,691 post 0/1 INFO :: Creating joint file ../../data/dedalus/spinup/output.h5
